<a href="https://colab.research.google.com/github/keerthi165/depthEstimation/blob/master/depthEstimationAttn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import h5py
from sklearn.model_selection import train_test_split

def featureTransferBlock(x):
  x_1 = tf.keras.layers.Conv2D(128,(1,1),padding='same')(x)
  x_2 = tf.keras.layers.Conv2D(128,(3,3),padding='same')(x_1)
  x_2 = tf.keras.layers.BatchNormalization()(x_2)
  x_2 = tf.keras.layers.ReLU()(x_2)
  x_2 = tf.keras.layers.Conv2D(128,(3,3),padding='same')(x_2)
  y = tf.keras.layers.add([x_1,x_2])
  y = tf.keras.layers.ReLU()(y)
  return y  

def AttentionBasedFeatureAgg(x,y):
  x = tf.keras.layers.UpSampling2D()(x)
  x_1 = tf.keras.layers.concatenate([x,y])
  x_1 = tf.keras.layers.MaxPool2D()(x_1)
  x_1 = tf.keras.layers.ReLU()(x_1)
  x_1 = tf.keras.layers.Activation(activation='sigmoid')(x_1)
  x_1 = tf.keras.layers.UpSampling2D()(x_1)
  f = tf.keras.layers.concatenate([x,x_1])
  return f

def featureFusion(x,y):
  x_1 = featureTransferBlock(x)
  x_1 = AttentionBasedFeatureAgg(x_1,y)
  x_1 = featureTransferBlock(x_1)
  x_1 = tf.keras.layers.UpSampling2D()(x_1)
  return x_1



hf = h5py.File('/content/drive/My Drive/nyuDataset/dataset.h5','r')
images = hf.get('images')
depths = hf.get('depths')

x_train,x_test,y_train,y_test = train_test_split(images,depths,test_size=0.33,shuffle=False)

input = tf.keras.layers.Input(shape=(480,640,3))
x_1 = tf.keras.layers.Conv2D(64,(1,1),padding = 'same')(input)
# x_1 = tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides = (1,1))(x_1)
x_1 = tf.keras.layers.Conv2D(64,(1,1),padding='same')(x_1)
x_1 = tf.keras.layers.Conv2D(64,(3,3),padding='same')(x_1)
x_1 = tf.keras.layers.Conv2D(128,(1,1),padding='same')(x_1)

x_2 = tf.keras.layers.Conv2D(256,(1,1),padding='same')(x_1)
x_2 = tf.keras.layers.Conv2D(256,(3,3),padding='same')(x_2)
x_2 = tf.keras.layers.Conv2D(512,(1,1),padding='same')(x_2)
for i in range(3):
  x_2 = tf.keras.layers.Conv2D(256,(1,1),padding='same')(x_2)
  x_2 = tf.keras.layers.Conv2D(256,(3,3),padding='same')(x_2)
  x_2 = tf.keras.layers.Conv2D(512,(1,1),padding='same')(x_2)

x_3 = tf.keras.layers.Conv2D(512,(1,1),padding='same')(x_2)
x_3 = tf.keras.layers.Conv2D(512,(3,3),padding='same')(x_3)
x_3 = tf.keras.layers.Conv2D(1024,(1,1),padding='same')(x_3)
for i in range(3):
  x_3 = tf.keras.layers.Conv2D(512,(1,1),padding='same')(x_3)
  x_3 = tf.keras.layers.Conv2D(512,(3,3),padding='same')(x_3)
  x_3 = tf.keras.layers.Conv2D(1024,(1,1),padding='same')(x_3)

x_4 = tf.keras.layers.Conv2D(1024,(1,1),padding='same')(x_3)
x_4 = tf.keras.layers.Conv2D(1024,(3,3),padding='same')(x_4)
x_4 = tf.keras.layers.Conv2D(2048,(1,1),padding='same')(x_4)
for i in range(2):
  x_4 = tf.keras.layers.Conv2D(1024,(1,1),padding='same')(x_4)
  x_4 = tf.keras.layers.Conv2D(1024,(3,3),padding='same')(x_4)
  x_4 = tf.keras.layers.Conv2D(1024,(1,1),padding='same')(x_4)  

x_5 = tf.keras.layers.Conv2D(256,(1,1))(x_4)
x_5 = tf.keras.layers.MaxPooling2D(pool_size=(1,1))(x_5)
x_5 = tf.keras.layers.UpSampling2D()(x_5)
x_5 = featureFusion(x_4,x_5)
x_3 = tf.keras.layers.UpSampling2D()(x_3)
x_5 = featureFusion(x_3,x_5)
for i in range(2):
  x_2 = tf.keras.layers.UpSampling2D()(x_2)
x_5 = featureFusion(x_2,x_5)
for i in range(3):
  x_1 = tf.keras.layers.UpSampling2D()(x_1)
x_5 = featureFusion(x_1,x_5)
x_5 = tf.keras.layers.Dropout(0.5)(x_5)
# x_5 = tf.keras.layers.Conv2D(512,(3,3),padding='same')(x_5)

for i in range(5):
  x_5 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x_5)

output = tf.keras.layers.Softmax()(x_5)
model = tf.keras.Model(inputs = input,outputs = output)
model.summary()
model.compile(optimizer = 'adam',loss='categorical_crossentropy',metrics = ['mse'])
y_pred = model.predict(x_test,10)
history = model.fit(x_train, y_train,validation_split = 0.25, epochs=8, batch_size= 1)
score, accu = model.evaluate(x_test,y_test)


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 480, 640, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 480, 640, 64) 256         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 480, 640, 64) 4160        conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 480, 640, 64) 36928       conv2d_1[0][0]                   
_______________________________________________________________________________________